# PyTorch to TensorFlow via ONNX

Sources:

- [Convert a PyTorch model to Tensorflow using ONNX](https://github.com/onnx/tutorials/blob/main/tutorials/PytorchTensorflowMnist.ipynb)
- [Pytorch to Tensorflow](https://github.com/entbappy/ONNX-Open-Neural-Network-Exchange/blob/master/Pytorch_to_Tensoflow/PyTorch%20to%20Tensorflow.ipynb)

In [9]:
# Install the specific package for ONNX-Tensorflow support
# Restart kernel
# !git clone https://github.com/onnx/onnx-tensorflow.git && cd onnx-tensorflow && pip install -e .
# Or, altenatively
# !pip install onnx_tf

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

import onnx
from onnx_tf.backend import prepare

c:\Users\Msagardi\AppData\Local\anaconda3\envs\ds\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


## PyTorch Model Definition

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [3]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} \tLoss: {:.6f}'.format(
                    epoch,  loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## Dataset: MNIST

In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



## Training

In [6]:
torch.manual_seed(1)
#device = torch.device("cuda")
device = torch.device("cpu")

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
 
for epoch in range(2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 0 	Loss: 2.371851

Test set: Average loss: 0.2103, Accuracy: 9399/10000 (94%)

Train Epoch: 1 	Loss: 0.380124

Test set: Average loss: 0.1260, Accuracy: 9611/10000 (96%)



## Save and Load Pytorch Model

In [8]:
# Save
torch.save(model.state_dict(), './output/model_mnist_pytorch.pth')

# Load
trained_model = Net()
trained_model.load_state_dict(torch.load('./output/model_mnist_pytorch.pth'))

<All keys matched successfully>

## Export to ONNX, Load and Visualize

In [28]:
dummy_input = Variable(torch.randn(1, 1, 28, 28)) 
torch.onnx.export(trained_model, dummy_input, "./output/model_mnist_pytorch.onnx")

In [2]:
# Load the ONNX file
model = onnx.load('./output/model_mnist_pytorch.onnx')

In [3]:
# Visualize
import netron

# Start visualization server
# Browser opens to visualize the ONNX model
# http://localhost:8080/
# We can click as see what's inside!
netron.start('output/model_mnist_pytorch.onnx')

Serving 'output/model_mnist_pytorch.onnx' at http://localhost:8080


('localhost', 8080)

In [4]:
netron.stop()

Stopping http://localhost:8080


In [7]:
input_nodes = [node.name for node in model.graph.input]
print(input_nodes)

['input.1']


## Inference with ONNX

In [5]:
import onnxruntime as rt
import numpy as np
from PIL import Image

#img = Image.open('./data/mnist_3.png').resize((28, 28)).convert('L')
img = Image.open('./data/mnist_7.png').resize((28, 28)).convert('L')
data = np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :]
print(data.shape)

sess = rt.InferenceSession('output/model_mnist_pytorch.onnx')
input_name = sess.get_inputs()[0].name # it is called input.1
label_name = sess.get_outputs()[0].name # it is called 20

pred_onnx = sess.run([label_name], {input_name: data})[0][0]
print(np.argmax(pred_onnx))

(1, 1, 28, 28)
7


## Convert ONNX to Tensorflow

In [11]:
# Import the ONNX model to Tensorflow
tf_rep = prepare(model)

In [12]:
# Input nodes to the model
print('inputs:', tf_rep.inputs)

# Output nodes from the model
print('outputs:', tf_rep.outputs)

# All nodes in the model
print('tensor_dict:')
print(tf_rep.tensor_dict)

inputs: ['input.1']
outputs: ['20']
tensor_dict:
{}


## Inference with Tensorflow

Unfortunately, the inference with Tensorflow doesn't work, because the seems to be a miss-match between the expected and actual input node names. I don't have t ime to fix this at the moment...

In [13]:
import numpy as np
from IPython.display import display
from PIL import Image

print('Image 1:')
img = Image.open('./data/mnist_3.png').resize((28, 28)).convert('L')
display(img)
output = tf_rep.run(np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :])
print('The digit is classified as ', np.argmax(output))

print('Image 2:')
img = Image.open('./data/mnist_7.png').resize((28, 28)).convert('L')
display(img)
output = tf_rep.run(np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :])
print('The digit is classified as ', np.argmax(output))

Image 1:


KeyError: in user code:

    File "c:\Users\Msagardi\AppData\Local\anaconda3\envs\ds\lib\site-packages\onnx_tf\backend_tf_module.py", line 99, in __call__  *
        output_ops = self.backend._onnx_node_to_tensorflow_op(onnx_node,
    File "c:\Users\Msagardi\AppData\Local\anaconda3\envs\ds\lib\site-packages\onnx_tf\backend.py", line 347, in _onnx_node_to_tensorflow_op  *
        return handler.handle(node, tensor_dict=tensor_dict, strict=strict)
    File "c:\Users\Msagardi\AppData\Local\anaconda3\envs\ds\lib\site-packages\onnx_tf\handlers\handler.py", line 59, in handle  *
        return ver_handle(node, **kwargs)
    File "c:\Users\Msagardi\AppData\Local\anaconda3\envs\ds\lib\site-packages\onnx_tf\handlers\backend\conv.py", line 15, in version_11  *
        return cls.conv(node, kwargs["tensor_dict"])
    File "c:\Users\Msagardi\AppData\Local\anaconda3\envs\ds\lib\site-packages\onnx_tf\handlers\backend\conv_mixin.py", line 29, in conv  *
        x = input_dict[node.inputs[0]]

    Key